In [72]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import random
from matplotlib.patches import Rectangle
from lxml import etree
# Enable Matplotlib backend for displaying static images
%matplotlib inline

In [73]:
# Subfolders for the class labels
import os
os.listdir("data/images/images")

['crease',
 'crescent_gap',
 'inclusion',
 'oil_spot',
 'punching_hole',
 'rolled_pit',
 'silk_spot',
 'waist folding',
 'water_spot',
 'welding_line']

In [74]:
# Annotations of the class labels
os.listdir("data/label/label")[:5]

['img_01_3402617700_00001.xml',
 'img_01_3402617700_01009.xml',
 'img_01_3436789500_00004.xml',
 'img_01_3436814600_00259.xml',
 'img_01_3436815300_00473.xml']

In [75]:
# Read all the images and count the number of image paths in the training subfolders
image_path = glob.glob("data/images/images/*/*.jpg")
# Normalize slashes to one format first
image_path = [p.replace('\\', '/') for p in image_path]

In [76]:
len(image_path)

2306

In [77]:
# Read all the annotation files and count the number of annotations in the training label folder
xmls_path = glob.glob("data/label/label/*.xml")
# Normalize slashes to one format first
xmls_path = [p.replace('\\', '/') for p in xmls_path]

In [78]:
len(xmls_path)

2280

In [151]:
# Sort the annotation and image files in ascending order
xmls_path.sort(key = lambda x:x.split("/")[-1].split(".xml")[0])
image_path.sort(key = lambda x:x.split("/")[-1].split(".jpg")[0])

In [152]:
xmls_path[:5]

['data/label/label/img_01_3402617700_00001.xml',
 'data/label/label/img_01_3402617700_01009.xml',
 'data/label/label/img_01_3436789500_00004.xml',
 'data/label/label/img_01_3436814600_00259.xml',
 'data/label/label/img_01_3436815300_00473.xml']

In [153]:
image_path[:5]

['data/images/images/crescent_gap/img_01_3402617700_00001.jpg',
 'data/images/images/crescent_gap/img_01_3402617700_01009.jpg',
 'data/images/images/crease/img_01_3436789500_00004.jpg',
 'data/images/images/silk_spot/img_01_3436814600_00259.jpg',
 'data/images/images/silk_spot/img_01_3436815300_00473.jpg']

In [81]:
# Extract the annotation filename without file extension
xmls_train = [path.split("/")[-1].split(".")[0] for path in xmls_path]

In [82]:
xmls_train[:5]

['img_01_3402617700_00001',
 'img_01_3402617700_01009',
 'img_01_3436789500_00004',
 'img_01_3436814600_00259',
 'img_01_3436815300_00473']

In [83]:
# Retrieve the image filename that have a corresponding annotation file.
# This will be the training set of images.
imgs_train = [img for img in image_path if (img.split("/")[-1].split)(".jpg")[0] in xmls_train]
imgs_train[:5]

['data/images/images/crescent_gap/img_01_3402617700_00001.jpg',
 'data/images/images/crescent_gap/img_01_3402617700_01009.jpg',
 'data/images/images/crease/img_01_3436789500_00004.jpg',
 'data/images/images/silk_spot/img_01_3436814600_00259.jpg',
 'data/images/images/silk_spot/img_01_3436815300_00473.jpg']

In [84]:
# Ensure equal counts of training images with the annotations
len(imgs_train),len(xmls_path)

(2280, 2280)

In [85]:
#########################
# SET UP THE DATA FRAME #
#########################

In [86]:
# Extract class label from the path of each image file
labels = [label.split("/")[-2] for label in imgs_train]
labels[:5]

['crescent_gap', 'crescent_gap', 'crease', 'silk_spot', 'silk_spot']

In [87]:
# Convert the labels into a Pandas DataFrame for the class labels and call them Defect Types
labels = pd.DataFrame(labels, columns = ["Defect Type"])
labels

,Defect Type
0,crescent_gap
1,crescent_gap
2,crease
3,silk_spot
4,silk_spot
...,...
2275,silk_spot
2276,silk_spot
2277,silk_spot
2278,silk_spot


In [88]:
# Set up a One-hot encoding to represent the defect types.  A one-hot encoded vector is a way of representing categorical values as binary vectors
# where exactly one element is 1 (the "hot" one) and all others are 0.  Each defect type is assigned a binary vector of 10 with the bit for the
# specific defect type turned "on".
from sklearn.preprocessing import LabelBinarizer

In [89]:
# Obtain training labels without duplication
Class = labels["Defect Type"].unique()

In [90]:
print(Class)

['crescent_gap' 'crease' 'silk_spot' 'water_spot' 'welding_line'
 'inclusion' 'oil_spot' 'waist folding' 'rolled_pit' 'punching_hole']


In [91]:
# Pair each element in Class with a corresponding number starting from 1
zip_result = list(zip(Class, range(1, len(Class) + 1)))
print(zip_result)

[('crescent_gap', 1), ('crease', 2), ('silk_spot', 3), ('water_spot', 4), ('welding_line', 5), ('inclusion', 6), ('oil_spot', 7), ('waist folding', 8), ('rolled_pit', 9), ('punching_hole', 10)]


In [92]:
# Transform the tuple into a map of defect type to it's numerical reference (defect class index)
Class_dict = dict(zip_result)
Class_dict

{'crescent_gap': 1,
 'crease': 2,
 'silk_spot': 3,
 'water_spot': 4,
 'welding_line': 5,
 'inclusion': 6,
 'oil_spot': 7,
 'waist folding': 8,
 'rolled_pit': 9,
 'punching_hole': 10}

In [93]:
# Create a new column called "Class" in the data frome and store the corresponding defect index
labels["Class"] = labels["Defect Type"].apply(lambda x: Class_dict[x])
labels

,Defect Type,Class
0,crescent_gap,1
1,crescent_gap,1
2,crease,2
3,silk_spot,3
4,silk_spot,3
...,...,...
2275,silk_spot,3
2276,silk_spot,3
2277,silk_spot,3
2278,silk_spot,3


In [94]:
# Convert the class dictionary into binary vectors for easier processing
lb = LabelBinarizer()
# Fit it to the class dictionary
lb.fit(list(Class_dict.values()))
# Convert multi-class labels to binary labels (belong or does not belong) to the labels
transformed_labels = lb.transform(labels["Class"])
transformed_labels[:5]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [95]:
transformed_labels.shape

(2280, 10)

In [96]:
# Create some new columns corresponding to the class index
y_bin_labels = [] 
for i in range(transformed_labels.shape[1]):
    y_bin_labels.append("Class" + str(i))
    # Add the columns to the data frame.
    labels["Class" + str(i + 1)] = transformed_labels[:, i]

In [97]:
y_bin_labels[:5]

['Class0', 'Class1', 'Class2', 'Class3', 'Class4']

In [98]:
labels

,Defect Type,Class,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10
0,crescent_gap,1,1,0,0,0,0,0,0,0,0,0
1,crescent_gap,1,1,0,0,0,0,0,0,0,0,0
2,crease,2,0,1,0,0,0,0,0,0,0,0
3,silk_spot,3,0,0,1,0,0,0,0,0,0,0
4,silk_spot,3,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2275,silk_spot,3,0,0,1,0,0,0,0,0,0,0
2276,silk_spot,3,0,0,1,0,0,0,0,0,0,0
2277,silk_spot,3,0,0,1,0,0,0,0,0,0,0
2278,silk_spot,3,0,0,1,0,0,0,0,0,0,0


In [99]:
# Drop any columns not needed for numerical processing
labels.drop("Class", axis = 1, inplace = True)
labels.drop("Defect Type", axis = 1, inplace = True)

In [102]:
# Display the current data frame
labels

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2275,0,0,1,0,0,0,0,0,0,0
2276,0,0,1,0,0,0,0,0,0,0
2277,0,0,1,0,0,0,0,0,0,0
2278,0,0,1,0,0,0,0,0,0,0


In [103]:
# Function to parse and extract information from annotation files and normalize coordinates between 0,1
# Relative bounding box coordinates express positions and dimensions as fractions of the image size, making them flexible and resolution-independent.
def to_labels(path):
    # Read the annotation file
    xml = open("{}".format(path)).read()                         
    sel = etree.HTML(xml)
    # Obtain the image width
    width = int(sel.xpath("//size/width/text()")[0])
    # Obtain the image height
    height = int(sel.xpath("//size/height/text()")[0])  
    # Extract the bounding box coordinates
    xmin = int(sel.xpath("//bndbox/xmin/text()")[0])
    xmax = int(sel.xpath("//bndbox/xmax/text()")[0])
    ymin = int(sel.xpath("//bndbox/ymin/text()")[0])
    ymax = int(sel.xpath("//bndbox/ymax/text()")[0])
    # Return the relative coordinates
    return [xmin/width, ymin/height, xmax/width, ymax/height]

In [108]:
# Display the relative bounding box coordinates for the annotations
coors = [to_labels(path) for path in xmls_path]
coors[:5]

[[0.8486328125, 0.806, 0.951171875, 0.993],
 [0.857421875, 0.298, 0.96044921875, 0.858],
 [0.47900390625, 0.182, 0.9990234375, 0.249],
 [0.3642578125, 0.017, 0.49365234375, 0.993],
 [0.3974609375, 0.017, 0.7431640625, 0.978]]

In [120]:
#Create a list of tuples.  Each tuple represents all the values in the annotations for a particular coordinate
xmin, ymin, xmax, ymax = list(zip(*coors))

In [119]:
xmin[:5], ymin[:5], xmax[:5], ymax[:5]

((0.8486328125, 0.857421875, 0.47900390625, 0.3642578125, 0.3974609375),
 (0.806, 0.298, 0.182, 0.017, 0.017),
 (0.951171875, 0.96044921875, 0.9990234375, 0.49365234375, 0.7431640625),
 (0.993, 0.858, 0.249, 0.993, 0.978))

In [124]:
# Convert to Numpy array.  Including the data frame values
xmin = np.array(xmin)
ymin = np.array(ymin)
xmax = np.array(xmax)
ymax = np.array(ymax)
label = np.array(labels.values)

In [125]:
xmin[:5], ymin[:5], xmax[:5], ymax[:5], label[:5]

(array([0.84863281, 0.85742188, 0.47900391, 0.36425781, 0.39746094]),
 array([0.806, 0.298, 0.182, 0.017, 0.017]),
 array([0.95117188, 0.96044922, 0.99902344, 0.49365234, 0.74316406]),
 array([0.993, 0.858, 0.249, 0.993, 0.978]),
 array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]))

In [157]:
# Creates a Dataset whose elements are slices of the given tensors
# Slicing a 1D tensor produces scalar tensor elements

# A tensor is a fundamental data structure in machine learning and deep learning — it's essentially 
# a multi-dimensional array that generalizes scalars, vectors, and matrices.

# A tensor is like a NumPy ndarray or a general-purpose container for numbers.
# You can think of it as a box that can hold numbers in any number of dimensions.

# Sliced in parallel into a data set of 5 elements
# (0.84863281, 0.806, 0.95117188, 0.993, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# (0.85742188, 0.298, 0.96044922, 0.858, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# ...

# The code builds a TensorFlow dataset where each element is a 5-tuple representing a bounding box and its label, 
# by slicing across the input sequences in parallel.

labels_dataset = tf.data.Dataset.from_tensor_slices((xmin, ymin, xmax, ymax, label))
for img in labels_dataset.take(1):
    print(img)

(<tf.Tensor: shape=(), dtype=float64, numpy=0.8486328125>, <tf.Tensor: shape=(), dtype=float64, numpy=0.806>, <tf.Tensor: shape=(), dtype=float64, numpy=0.951171875>, <tf.Tensor: shape=(), dtype=float64, numpy=0.993>, <tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])>)


In [144]:
# Read a .jpg image from disk, resize it to 224x224 pixels, convert to format that TensorFlow can use from
# training and normalize the pixel values to be between 0 and 1.
def load_image(path):
    # read raw bytes
    image = tf.io.read_file(path)
    # decode the image bytes into a usable image format and ensure it has 3 color channels (red, green, blue)
    image = tf.image.decode_jpeg(image,3)       
    # resize to 224x224 - common size  used in many pretrained models
    image = tf.image.resize(image,[224,224])
    # converts the pixel values to 32-bit floating point numbers (from integers).  Neural networks typically expect inputs
    # as floating-point numbers also, when dividing by 255 we don't to truncate to 0 or 1.
    image = tf.cast(image,tf.float32)  
    image = image / 255
    # normalize pixel values from [0-255] to [0.0-1.0]
    return image

In [142]:
imgs_train[:1]

['data/images/images/crescent_gap/img_01_3402617700_00001.jpg']

In [149]:
# For feeding images into a convolutional neural network (CNN) for classification or detection
# create a dataset where each item is an image tensor of shape (224, 224, 3) — a color image — and the pixel values are of type float32.

# Takes a list of image file paths and creates a dataset where each elelment is a file path string
dataset = tf.data.Dataset.from_tensor_slices(imgs_train)
# each element in the dataset is a preprocessed image tensor (224 x 224 pixels with 3 color channels)
dataset = dataset.map(load_image)
dataset

<_MapDataset element_spec=TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None)>

In [162]:
# Create tuples from the images and image labels data sets
dataset_label = tf.data.Dataset.zip((dataset, labels_dataset))
dataset_label
#for img in dataset_label.take(1):
    #print(img)

<_ZipDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None)))>